<h1>Задание 3<h1>

Собрать корпус документов по выбранной теме и по основы программ, разработанных в заданиях 1 и 2, подсчитать метрику TF-IDF для ключевых слов документа. (2 балла)

<h2>Стемматизация<h2>

In [1]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import Stemmer
stemmer = Stemmer.Stemmer('russian')

#Create and stopwords list
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на', 'как-то'])

In [2]:
punctuation = punctuation + '...' + '—' + '«' + '»' + '``\n'
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~...—«»``\n'

In [5]:
def get_sentences(text):
    split_regex = re.compile(r'[.|!|?|…]')
    sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(text)])
    corpus = []
    for s in sentences:
        corpus.append(s)
    return(corpus)

In [6]:
russian_stopwords_lem = stemmer.stemWords(russian_stopwords)
#Preprocess function
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens]
    stemmed_tokens = stemmer.stemWords(tokens)
    filtered_tokens = []
    for word in stemmed_tokens:
        if (word not in russian_stopwords_lem) and (word != " ") and (word not in punctuation):
            filtered_tokens.append(word)
    
    
    return filtered_tokens

In [7]:
from nltk import FreqDist
def frequency_of_words(txt):
    t = preprocess_text(txt)
    frequency_distribution = FreqDist(t)
    return frequency_distribution

In [8]:
def write_to_file(frequency_distribution, name):
    import csv
    name = str(name).replace('corpus_/','')
    path = 'words_frequency/' + name[:-4] + '.csv'
    with open(path, 'w') as csvfile:
        fieldnames = ['word', 'frequency']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for key in frequency_distribution:
             writer.writerow({'word': key, 'frequency': str(frequency_distribution[key])})
                                  
        
    csvfile.close()

    print("Writing complete")

In [9]:
import os
from pathlib import Path
source_dir = Path('corpus_/')
files = source_dir.glob('*.txt')
for file in files:
    txt = ''
    with file.open('r') as file_handle:
        for line in file_handle:
            txt += line.rstrip('\r\n') + ' '
    
    txt = get_sentences(txt)
    txt = ' '.join(txt)
    fr = frequency_of_words(txt)
    write_to_file(fr, file)
    print(file, ' done')

Writing complete
corpus_/Кино - Город.txt  done
Writing complete
corpus_/Кино - Электричка.txt  done
Writing complete
corpus_/Кино - Мои друзья.txt  done
Writing complete
corpus_/Кино - Разреши мне.txt  done
Writing complete
corpus_/Кино - В поисках сюжета.txt  done
Writing complete
corpus_/Кино - Малыш.txt  done
Writing complete
corpus_/Кино - Спокойная ночь.txt  done
Writing complete
corpus_/Кино - Группа крови.txt  done
Writing complete
corpus_/Кино - Пачка сигарет.txt  done
Writing complete
corpus_/Кино - Война.txt  done
Writing complete
corpus_/Кино - Место для шага вперед.txt  done
Writing complete
corpus_/Кино - Ночь.txt  done
Writing complete
corpus_/Кино - Верь мне.txt  done
Writing complete
corpus_/Кино - На кухне.txt  done
Writing complete
corpus_/Кино - Без десяти.txt  done
Writing complete
corpus_/Кино - Песня без слов.txt  done
Writing complete
corpus_/Кино - Вопрос.txt  done
Writing complete
corpus_/Кино - Ты обвела меня вокруг пальца.txt  done
Writing complete
corpus_/К

**TF термина а = (Количество раз, когда термин а встретился в тексте / количество всех слов в тексте)**

**IDF термина а = логарифм(Общее количество документов / Количество документов, в которых встречается термин а)**

**TF-IDF термина а = (TF термина а) * (IDF термина а)**

In [15]:
import pandas as pd

main_text = pd.read_csv('words_frequency/Кино - Перемен.csv')

all_words = main_text['frequency'].sum()
main_text['TF'] = main_text['frequency']/all_words

In [16]:
def compute_in_other_texts(word, corpus):
    return sum([1.0 for i in corpus if word in i])

def compute_idf(word, corpus):
        return math.log10(len(corpus)/sum([1.0 for i in corpus if word in i]))

In [17]:
import math
corpus = [0] * 68
source_dir = Path('words_frequency/')
files = source_dir.glob('*.csv')
main_list = list(main_text['word'])

main_text['in_other_texts'] = 0
main_text['IDF'] = 0

i=0
for file in files:
    text = pd.read_csv(file)
    text_list = list(text['word'])
    corpus[i] = text_list
    i+=1

i=0
for word in main_list:
    main_text.iloc[i,3] = compute_in_other_texts(word, corpus)
    i+=1

i=0
for word in main_list:
    main_text.iloc[i,4] = compute_idf(word, corpus)
    i+=1

main_text['TF-IDF'] = main_text['TF']*main_text['IDF']

In [18]:
main_text.shape

(61, 6)

если слово встречается в каком-либо документе часто, при этом встречаясь редко во всех остальных документах — это слово имеет большую значимость для того самого документа

In [21]:
main_text.sort_values('TF-IDF', ascending=False).head(60)

,word,frequency,TF,in_other_texts,IDF,TF-IDF
1,перем,12,0.106195,1,1.832509,0.194603
2,треб,6,0.053097,1,1.832509,0.097301
0,наш,13,0.115044,12,0.753328,0.086666
6,смех,3,0.026549,1,1.832509,0.048651
7,слез,3,0.026549,1,1.832509,0.048651
8,пульсац,3,0.026549,1,1.832509,0.048651
5,сердц,3,0.026549,2,1.531479,0.040659
9,вен,3,0.026549,2,1.531479,0.040659
10,ждем,3,0.026549,2,1.531479,0.040659
3,глаз,4,0.035398,10,0.832509,0.029469
